# **Problem: MobileNetV2 in tensorflow. Train on cats and dogs dataset.**

Python program to train cats and dogs dataset using MobileNetV2 in tensorflow (without using transfer learning).

Run all the cells. After executing the last cell, you will get the accuracy of the trained model.

**Notes:**

Following things are needed to be checked before running the program.
  1. Tensorflow module is needed to be installed.
  2. Zip file module is needed to be installed.
  3. gdown module is needed to be installed.
  4. Check whether you have given the correct location of your dataset file.
  5. You should have access to the file in the Google Drive.
  

# **Import Modules**

In [1]:
# Import tensorflow
import tensorflow as tf

# Import keras
from tensorflow import keras

# Import preprocess_input for ImageDataGenerator class
from keras.applications.mobilenet import preprocess_input

# Import ImageDataGenerator class for the Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

# Import gdown module to download files from google drive
import gdown

# **Get the file location from google drive**

In [ ]:
# Please change the URL as needed (make sure you have the access to the file)

url = 'https://drive.google.com/file/d/1fMHrqIY0QYEj9qFUFsDuF949Jo-UWzVX/view?usp=sharing'

# Derive the file id from the URL
file_id = url.split('/')[-2]

# Derive the download url of the the file
download_url = 'https://drive.google.com/uc?id=' + file_id

# Give the location you want to save it in your local machine
file_location = r'cats_and_dogs.zip'

# Download the file from drive to your local machine
gdown.download(download_url, file_location, quiet=False)


# **Unzip the zip dataset**

In [ ]:
!unzip /content/cats_and_dogs.zip -d "/content/unzipped_folder/"

# **Construct train and test datasets**

In [4]:
train_path = r"/content/unzipped_folder/test_set/test_set"
test_path = r"/content/unzipped_folder/training_set/training_set"

# Give image size and shape
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Go through the train directory to obtain cateogries
train_batches = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
                train_path ,target_size=(IMG_SIZE,IMG_SIZE),batch_size=24,class_mode='categorical')

# Go through the test directory to obtain cateogries
test_batches = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(
                test_path ,target_size=(IMG_SIZE,IMG_SIZE),batch_size=24,class_mode='categorical')

Found 55 images belonging to 2 classes.
Found 206 images belonging to 2 classes.


# **Derive the model using MobileNetV2**

In [5]:
# Get the base model using MobileNetV2 without a pre trained moded(weight = None)
# Do not make trainable as False (Keras is trainable is true by default)
base_model = keras.applications.MobileNetV2(input_shape=IMG_SHAPE,include_top=False,weights=None)

feature_batch = base_model.output
global_average_layer = keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_layer = keras.layers.Dense(2)
prediction_batch = prediction_layer(feature_batch_average)

# Group layers into a keras model
model = keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

# **Compile and train the model**

In [ ]:
# Complile the model
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_batches,
                         steps_per_epoch=24,
                         epochs=1, #<-- increase for higher accuracy
                         validation_data=test_batches,
                         validation_steps=200)

# **Get accuracy of the model**

In [ ]:
# Get the accuracy
loss,accuracy = model.evaluate(test_batches, steps = 20)

print('Accuracy :', accuracy)